In [ ]:
import cv2
import numpy as np
from skimage.metrics import structural_similarity as ssim # install scikit-image
from sklearn.cluster import KMeans # install scikit-learn

In [ ]:
# read images and convert to RGB
## image 1 = generated image
## image 2 = real image

image1 = cv2.imread("ny-m-epoch207_fake_B.png")
image1 = cv2.cvtColor(image1, cv2.COLOR_BGR2RGB)

image2 = cv2.imread("ny-m-epoch207_real_B.png")
image2 = cv2.cvtColor(image2, cv2.COLOR_BGR2RGB)

In [ ]:
# pixel count for specific color range

# Define the color range
lower_red = np.array([0, 0, 100])
upper_red = np.array([50, 56, 255])

# Create a mask that only includes the defined color range
mask = cv2.inRange(image1, lower_red, upper_red)

# Count the pixels
red_pixel_count = cv2.countNonZero(mask)

print(f'The number of red pixels is: {red_pixel_count}')

In [ ]:
# pixel count for all colors

# Flatten the image array to 2D, each row is a pixel, with columns for B, G, and R values
pixels = image1.reshape(-1, 3)

# Convert pixels to a more manageable type
pixels = np.array(pixels, dtype=np.uint8)

# Find unique colors and their counts
unique_colors, counts = np.unique(pixels, axis=0, return_counts=True)

# Print or process the unique colors and their counts
for color, count in zip(unique_colors, counts):
    print(f"Color: {color}, Count: {count}")

In [ ]:
# quantize the colors

def quantize_image(image, n_colors=8):
    # Reshape the image to a 2D array of pixels
    pixels = image.reshape(-1, 3)
    
    # Use KMeans clustering to find the most dominant colors
    kmeans = KMeans(n_clusters=n_colors, random_state=0).fit(pixels)
    
    # Map each pixel to the nearest color centroid
    new_pixels = kmeans.cluster_centers_[kmeans.labels_]
    
    # Reshape back to the original image shape
    quantized_image = new_pixels.reshape(image.shape).astype('uint8')
    
    return quantized_image, kmeans.cluster_centers_

In [ ]:
# count the pixels of each quantized color

def count_pixels(image, centroids):
    unique, counts = np.unique(image.reshape(-1, image.shape[2]), axis=0, return_counts=True)
    color_counts = {tuple(centroids[i]): count for i, count in enumerate(counts)}
    return color_counts

In [ ]:
# Quantize the images
quantized_image1, centroids1 = quantize_image(image1, n_colors=8)
quantized_image2, centroids2 = quantize_image(image2, n_colors=8)

# Count the pixels for each quantized color
color_counts1 = count_pixels(quantized_image1, centroids1)
color_counts2 = count_pixels(quantized_image2, centroids2)

# Compare the pixel counts
# This is a simple comparison; for more detailed analysis, you may need to align the centroids
# and compare the counts for each matched color.
print("Image 1 Color Counts:")
for color, count in color_counts1.items():
    print(f"Color: {color}, Count: {count}")

print("\nImage 2 Color Counts:")
for color, count in color_counts2.items():
    print(f"Color: {color}, Count: {count}")

In [ ]:
# Mean Squared Error (MSE)
## MSE calculates the average squared difference between the pixel intensities of two images. 
## It gives a single number that represents the error between the two images; the lower the MSE, the more similar the images are.
## MSE is very sensitive to even slight changes in pixel values.

def mse(imageA, imageB):
    # The MSE between the two images is the sum of the squared difference divided by the number of pixels
    err = np.sum((imageA.astype("float") - imageB.astype("float")) ** 2)
    err /= float(imageA.shape[0] * imageA.shape[1])
    
    return err

# Ensure the images are the same size
image1 = cv2.resize(image1, (image2.shape[1], image2.shape[0]))

# Compare the images
print(f"MSE: {mse(image1, image2)}")

In [ ]:
# Structural Similarity Index (SSIM)
## SSIM measures the similarity between two images in terms of luminance, contrast, and structure. 
## other than MSE, SSIM can be more aligned with human visual perception. 
## The SSIM index is a decimal value between -1 and 1; a value of 1 indicates perfect similarity.

def compare_images(imageA, imageB):
    # Compute SSIM between two images
    s = ssim(imageA, imageB, multichannel=True)
    
    return s

# Convert the images to grayscale
image1_gray = cv2.cvtColor(image1, cv2.COLOR_BGR2GRAY)
image2_gray = cv2.cvtColor(image2, cv2.COLOR_BGR2GRAY)

# Ensure the images are the same size
image1_gray = cv2.resize(image1_gray, (image2_gray.shape[1], image2_gray.shape[0]))

# Compare the images
print(f"SSIM: {compare_images(image1_gray, image2_gray)}")

In [ ]:
# Feature matching 
## Feature matching techniques, such as SIFT (Scale-Invariant Feature Transform), SURF (Speeded-Up Robust Features), or ORB (Oriented FAST and Rotated BRIEF), detect and describe local features in images. 
## These features can then be matched across images to find similarities, even in the presence of scale, rotation, and illumination changes.

# Initialize ORB detector
orb = cv2.ORB_create()

# Find the keypoints and descriptors with ORB
kp1, des1 = orb.detectAndCompute(image1, None)
kp2, des2 = orb.detectAndCompute(image2, None)

# Create BFMatcher object
bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)

# Match descriptors
matches = bf.match(des1, des2)

# Sort them in the order of their distance
matches = sorted(matches, key=lambda x:x.distance)

# Draw first 10 matches
result = cv2.drawMatches(image1, kp1, image2, kp2, matches[:10], None, flags=2)

# Display the result
cv2.imshow("Matches", result)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
# Initialize SIFT detector
## look into tolerance
sift = cv2.SIFT_create()

# Detect and compute the descriptors with SIFT
keypoints1, descriptors1 = sift.detectAndCompute(image1, None)
keypoints2, descriptors2 = sift.detectAndCompute(image2, None)

# Create BFMatcher object
bf = cv2.BFMatcher(cv2.NORM_L2, crossCheck=True)

# Match descriptors
matches = bf.match(descriptors1, descriptors2)

# Sort them in the order of their distance
matches = sorted(matches, key=lambda x:x.distance)

# Draw first 10 matches
img_matches = cv2.drawMatches(image1, keypoints1, image2, keypoints2, matches[:10], None, flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)

# Display the matches
cv2.imshow("SIFT Matches", img_matches)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
# Initialize SURF detector
surf = cv2.xfeatures2d.SURF_create(400)  # Threshold for hessian keypoint detector

# Detect and compute the descriptors with SURF
keypoints1, descriptors1 = surf.detectAndCompute(image1, None)
keypoints2, descriptors2 = surf.detectAndCompute(image2, None)

# Create BFMatcher object
bf = cv2.BFMatcher(cv2.NORM_L2, crossCheck=True)

# Match descriptors
matches = bf.match(descriptors1, descriptors2)

# Sort them in the order of their distance
matches = sorted(matches, key=lambda x:x.distance)

# Draw first 10 matches
img_matches = cv2.drawMatches(image1, keypoints1, image2, keypoints2, matches[:10], None, flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)

# Display the matches
cv2.imshow("SURF Matches", img_matches)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
# Union over Intersection (IoU)
## IoU is a measure of the overlap between two images.

def quantize_image(image, n_colors=8):
    # Reshape the image to a 2D array of pixels
    pixels = image.reshape(-1, 3)
    
    # Use KMeans clustering to find the most dominant colors
    kmeans = KMeans(n_clusters=n_colors, random_state=0).fit(pixels)
    
    # Map each pixel to the nearest color centroid
    new_pixels = kmeans.cluster_centers_[kmeans.labels_]
    
    # Reshape back to the original image shape
    quantized_image = new_pixels.reshape(image.shape).astype('uint8')
    
    return quantized_image, kmeans.cluster_centers_

def count_pixels(image):
    # Flatten the image and convert to a tuple to make it hashable
    pixels = [tuple(pixel) for pixel in image.reshape(-1, 3)]
    # Count each unique color
    color_counts = {color: pixels.count(color) for color in set(pixels)}
    return color_counts

def calculate_uoi(color_counts1, color_counts2):
    colors1 = set(color_counts1.keys())
    colors2 = set(color_counts2.keys())
    
    intersection = colors1.intersection(colors2)
    union = colors1.union(colors2)
    
    uoi = len(intersection) / len(union) if len(union) > 0 else 0
    return uoi

# Quantize the images
quantized_image1, _ = quantize_image(image1, n_colors=8)
quantized_image2, _ = quantize_image(image2, n_colors=8)

# Count the pixels for each quantized color
color_counts1 = count_pixels(quantized_image1)
color_counts2 = count_pixels(quantized_image2)

# Calculate UoI
uoi = calculate_uoi(color_counts1, color_counts2)
print(f"Union over Intersection (UoI) for color comparison: {uoi}")


In [ ]:
## create and use mask for training dataset (black and white)
## single color for all parcels and a white background (for parcel dataset only)